In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# create directory for data
!mkdir data

# unzip files from zip folder
!unzip "/content/drive/MyDrive/PG/PB/Projekt badawczy - terminy medyczne/001_004.zip" -d "data/"

In [ ]:
data_directory = "./data/001_004/"

In [ ]:
import pathlib
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from pydub import AudioSegment, effects
import os

In [ ]:
data_path = pathlib.Path(data_directory)

files = list(data_path.glob('*.wav'))